
# 🌪️ Storm Damage Prediction — v5.6 (FAST, GPU‑aware)
Predict **Property** and **Crop** damages from NOAA Storm Events using:
- Text embeddings (SentenceTransformers, GPU‑aware)
- Numeric + Categorical features
- Geospatial & light spatiotemporal context (fast network features)
- **XGB Tweedie** with **Slim Optuna** tuning (GPU/CPU auto)
- **Quantile XGB** for prediction intervals
- **SHAP** for feature importance & explainability

**Runtime modes:** set env var `STORM_MODE` to `2h` (default), `3h`, or `full`.


In [1]:

# === 1) Setup & Utils ===
import os, sys, math, json, time, gc, warnings
from pathlib import Path
import numpy as np
import pandas as pd
from contextlib import contextmanager

warnings.filterwarnings("ignore")

SEED = 42
MODE = os.getenv("STORM_MODE", "2h").lower()
HAS_CUDA = False
try:
    import torch
    HAS_CUDA = torch.cuda.is_available()
except Exception:
    pass

OUTDIR = Path("./results"); OUTDIR.mkdir(parents=True, exist_ok=True)

@contextmanager
def timer(msg):
    import time
    t0 = time.time()
    print(f"⏱️ {msg} ...")
    try:
        yield
    finally:
        print(f"⏱️ {msg}: {time.time()-t0:.2f}s")

def save_json(obj, path):
    path = Path(path); path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f: json.dump(obj, f, indent=2)

def save_joblib(obj, path):
    import joblib
    path = Path(path); path.parent.mkdir(parents=True, exist_ok=True)
    joblib.dump(obj, path)

print(f"MODE={MODE} | CUDA={HAS_CUDA} | OUTDIR={OUTDIR.resolve()}")


MODE=2h | CUDA=True | OUTDIR=C:\Users\ducan\Documents\lmu\LMUmaster\sem3\sem3Projekte\DamagesPrediction5\results


In [2]:

# === 2) Load & Clean ===
from datetime import datetime

# Set your CSV path here or via env var STORM_CSV
CSV_PATH = os.environ.get("STORM_CSV", "./StormEvents_details-ftp_v1.0_d2013_c20250520.csv")

def parse_damage(v):
    if pd.isna(v): return np.nan
    s = str(v).strip().upper()
    if not s: return np.nan
    mult = 1
    if s.endswith('K'): mult, s = 1_000, s[:-1]
    elif s.endswith('M'): mult, s = 1_000_000, s[:-1]
    elif s.endswith('B'): mult, s = 1_000_000_000, s[:-1]
    try: return float(s) * mult
    except:
        try: return float(s.replace(',',''))
        except: return np.nan

df = pd.read_csv(CSV_PATH, low_memory=False, encoding='utf-8')
df.columns = [c.strip().upper() for c in df.columns]

assert 'DAMAGE_PROPERTY' in df.columns and 'DAMAGE_CROPS' in df.columns, "CSV missing target columns"

df['Y_PROP'] = df['DAMAGE_PROPERTY'].apply(parse_damage)
df['Y_CROP'] = df['DAMAGE_CROPS'].apply(parse_damage)
df = df[(df['Y_PROP'].notna()) | (df['Y_CROP'].notna())].copy()

# Dates & duration
for c in ['BEGIN_DATE_TIME','END_DATE_TIME']:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)
df['DURATION_HOURS'] = (df['END_DATE_TIME'] - df['BEGIN_DATE_TIME']).dt.total_seconds()/3600
df['DURATION_HOURS'] = df['DURATION_HOURS'].clip(lower=0).fillna(0)

# Geospatial numeric
for c in ['BEGIN_LAT','BEGIN_LON','END_LAT','END_LON']:
    if c in df.columns: df[c] = pd.to_numeric(df[c], errors='coerce')
df['LAT_MEAN'] = df[['BEGIN_LAT','END_LAT']].mean(axis=1)
df['LON_MEAN'] = df[['BEGIN_LON','END_LON']].mean(axis=1)

# Robust haversine distance (km) between begin and end points (fallback to 0 if missing)
def haversine_km(lat1, lon1, lat2, lon2):
    lat1 = np.deg2rad(lat1); lon1 = np.deg2rad(lon1)
    lat2 = np.deg2rad(lat2); lon2 = np.deg2rad(lon2)
    dlat = lat2 - lat1; dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
    return 6371.0 * c

df['EVENT_SPAN_KM'] = haversine_km(df['BEGIN_LAT'].fillna(df['LAT_MEAN']),
                                   df['BEGIN_LON'].fillna(df['LON_MEAN']),
                                   df['END_LAT'].fillna(df['LAT_MEAN']),
                                   df['END_LON'].fillna(df['LON_MEAN']))
df['EVENT_SPAN_KM'] = df['EVENT_SPAN_KM'].fillna(0).astype('float32')

# Drop high-cardinality / unused
for c in ['EPISODE_ID','EVENT_ID','DATA_SOURCE','BEGIN_RANGE','END_RANGE','BEGIN_AZIMUTH','END_AZIMUTH']:
    if c in df.columns: df.drop(columns=c, inplace=True)

print("Cleaned shape:", df.shape)
df.head(2)


Cleaned shape: (52259, 50)


,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,STATE,STATE_FIPS,YEAR,MONTH_NAME,...,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,Y_PROP,Y_CROP,DURATION_HOURS,LAT_MEAN,LON_MEAN,EVENT_SPAN_KM
0,201302,23,1900,201302,25,400,NEW HAMPSHIRE,33,2013,February,...,NaN,NaN,A coastal low moved southeast of southern New ...,Three to five inches of snow fell across easte...,0.0,0.0,33.0,NaN,NaN,0.0
1,201312,14,2100,201312,15,1300,NEW HAMPSHIRE,33,2013,December,...,NaN,NaN,"Low pressure moved out of the midwest, off the...",Eight to nine inches of snow fell across easte...,0.0,0.0,16.0,NaN,NaN,0.0


In [3]:

# === 3) Feature Lists & Base Matrix ===
text_cols = [c for c in ['EPISODE_NARRATIVE','EVENT_NARRATIVE'] if c in df.columns]
cat_cols  = [c for c in ['STATE','EVENT_TYPE','CZ_TYPE','CZ_NAME','LOCATION_NAME'] if c in df.columns]
num_cols  = [c for c in ['INJURIES_DIRECT','INJURIES_INDIRECT','DEATHS_DIRECT','DEATHS_INDIRECT',
                         'DURATION_HOURS','LAT_MEAN','LON_MEAN','EVENT_SPAN_KM'] if c in df.columns]

X_cols = num_cols + cat_cols + text_cols
df_model = df[X_cols + ['Y_PROP','Y_CROP','BEGIN_DATE_TIME']].copy()

# Targets: natural and log
y = np.column_stack([df_model['Y_PROP'].fillna(0).values, df_model['Y_CROP'].fillna(0).values])
y_log = np.log1p(y)

from sklearn.model_selection import train_test_split
X = df_model[X_cols].copy()
X_train, X_temp, y_train, y_temp, y_train_log, y_temp_log = train_test_split(
    X, y, y_log, test_size=0.30, random_state=SEED
)
X_valid, X_test, y_valid, y_test, y_valid_log, y_test_log = train_test_split(
    X_temp, y_temp, y_temp_log, test_size=0.50, random_state=SEED
)
print('Split sizes:', len(X_train), len(X_valid), len(X_test))


Split sizes: 36581 7839 7839


In [5]:
# === 3b) Spatiotemporal Network Features (FAST) — patched ===
from sklearn.neighbors import KDTree

print('🔧 Building spatiotemporal network features (fast)…')

df_net = pd.DataFrame(index=df_model.index)
df_net['BEGIN_DATE_TIME'] = df_model['BEGIN_DATE_TIME']
df_net['Y_PROP'] = df_model['Y_PROP']
df_net['Y_CROP'] = df_model['Y_CROP']
df_net['LAT_MEAN'] = df['LAT_MEAN'].values
df_net['LON_MEAN'] = df['LON_MEAN'].values

mask_valid = df_net['LAT_MEAN'].notna() & df_net['LON_MEAN'].notna()
coords = np.vstack([
    np.deg2rad(df_net.loc[mask_valid,'LAT_MEAN'].astype(float)),
    np.deg2rad(df_net.loc[mask_valid,'LON_MEAN'].astype(float))
]).T

nn_within_radius = np.zeros(len(df_net), dtype=np.float32)
recent_nn_6h = np.zeros(len(df_net), dtype=np.float32)
recent_nn_24h = np.zeros(len(df_net), dtype=np.float32)
recent_nn_72h = np.zeros(len(df_net), dtype=np.float32)
mean_damage_nearby = np.zeros(len(df_net), dtype=np.float32)
mean_damage_recent = np.zeros(len(df_net), dtype=np.float32)

if len(coords) > 0:
    tree = KDTree(coords, metric="euclidean")
    def km_to_rad(km): return km / 6371.0
    RADIUS_RAD = km_to_rad(100)

    dt = pd.to_datetime(df_net["BEGIN_DATE_TIME"], errors="coerce")
    event_hours = (dt.astype('int64') // 1_000_000_000) / 3600.0  # minor: avoid .view warning
    event_hours = event_hours.fillna(0).values.astype('float64')

    valid_idx = np.flatnonzero(mask_valid.values)
    for pos, i in enumerate(valid_idx):
        idx = tree.query_radius(coords[pos:pos+1], r=RADIUS_RAD)[0]
        neigh = valid_idx[idx]          # positional indices into df_net
        neigh = neigh[neigh != i]
        nn_within_radius[i] = len(neigh)
        if len(neigh) > 0:
            # nearby mean damage (use iloc with positional indices)  # FIX
            mean_damage_nearby[i] = np.nanmean(
                df_net.iloc[neigh][['Y_PROP','Y_CROP']].to_numpy()
            )
            # time filters
            td = event_hours[i] - event_hours[neigh]
            recent_nn_6h[i] = np.sum((0 < td) & (td <= 6))
            recent_nn_24h[i] = np.sum((0 < td) & (td <= 24))
            recent_nn_72h[i] = np.sum((0 < td) & (td <= 72))
            mask_r = (0 < td) & (td <= 72)
            if mask_r.any():
                # recent mean damage (use iloc)                       # FIX
                mean_damage_recent[i] = np.nanmean(
                    df_net.iloc[neigh[mask_r]][['Y_PROP','Y_CROP']].to_numpy()
                )

# Attach
extra_cols = ['NN_100KM','NN_6H','NN_24H','NN_72H','MEAN_DAMAGE_NEAR','MEAN_DAMAGE_RECENT']
df_model['NN_100KM'] = nn_within_radius
df_model['NN_6H'] = recent_nn_6h
df_model['NN_24H'] = recent_nn_24h
df_model['NN_72H'] = recent_nn_72h
df_model['MEAN_DAMAGE_NEAR'] = mean_damage_nearby
df_model['MEAN_DAMAGE_RECENT'] = mean_damage_recent

# Rebuild X with dedup protection
X = df_model[X_cols + extra_cols]
X = X.loc[:, ~X.columns.duplicated()]
for c in extra_cols:
    if c not in num_cols: num_cols.append(c)

# Re-split to align
X_train, X_temp, y_train, y_temp, y_train_log, y_temp_log = train_test_split(
    X, y, y_log, test_size=0.30, random_state=SEED
)
X_valid, X_test, y_valid, y_test, y_valid_log, y_test_log = train_test_split(
    X_temp, y_temp, y_temp_log, test_size=0.50, random_state=SEED
)
print('✅ Added network features. New split sizes:', len(X_train), len(X_valid), len(X_test))


🔧 Building spatiotemporal network features (fast)…
✅ Added network features. New split sizes: 36581 7839 7839


In [ ]:

# === 4) Preprocessing (GPU‑safe embeddings + memory‑safe OHE) ===
import os
os.environ.setdefault('TRANSFORMERS_NO_TORCHVISION','1')
from sentence_transformers import SentenceTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer

EMBED_MODEL_NAME = os.getenv("EMBED_MODEL_NAME", "sentence-transformers/all-MiniLM-L6-v2")
embed_model = SentenceTransformer(EMBED_MODEL_NAME, device=("cuda" if HAS_CUDA else "cpu"))
try:
    embed_model.max_seq_length = 256
except Exception:
    pass

def embed_text_batched(X, batch_size=512):
    if isinstance(X, pd.DataFrame):
        seq = X.iloc[:,0].astype(str).fillna('').tolist()
    elif isinstance(X, pd.Series):
        seq = X.astype(str).fillna('').tolist()
    else:
        seq = [str(t) for t in X]
    outputs=[]; i=0; bs=batch_size
    while i < len(seq):
        j = min(i+bs, len(seq)); chunk = seq[i:j]
        try:
            with torch.inference_mode():
                embs = embed_model.encode(chunk, batch_size=bs, convert_to_numpy=True, show_progress_bar=False, normalize_embeddings=False)
            outputs.append(embs.astype('float32', copy=False)); i=j
        except RuntimeError as e:
            if 'CUDA out of memory' in str(e) and bs>8 and HAS_CUDA:
                torch.cuda.empty_cache(); bs = max(8, bs//2)
                print(f"[embed_text] OOM → reducing batch_size to {bs}")
            else:
                raise
    if not outputs:
        dim = embed_model.get_sentence_embedding_dimension()
        return np.empty((0, dim), dtype=np.float32)
    return np.vstack(outputs)

num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Handle sklearn version diff for OHE
try:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=np.float32, max_categories=200)
except TypeError:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False, dtype=np.float32)

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', ohe)
])

transformers = []
if num_cols: transformers.append(('num', num_pipe, num_cols))
if cat_cols: transformers.append(('cat', cat_pipe, cat_cols))
for c in text_cols:
    transformers.append((f'text_{c}', FunctionTransformer(embed_text_batched, validate=False), [c]))

preprocess = ColumnTransformer(transformers=transformers, remainder='drop', n_jobs=1)
print(f'✅ Preprocess ready. Text model: {EMBED_MODEL_NAME} on {("cuda" if HAS_CUDA else "cpu")}')


In [ ]:

# === 5) XGB Tweedie (GPU‑aware) + Slim Optuna ===
import xgboost as xgb, optuna
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def sanitize_targets_nat(y_arr):
    y_arr = np.nan_to_num(y_arr, nan=0.0, posinf=0.0, neginf=0.0).astype('float32')
    y_arr[y_arr < 0] = 0.0
    return y_arr

def make_xgb_tweedie(base_params=None):
    params = dict(objective='reg:tweedie', tweedie_variance_power=1.4,
                  tree_method='hist', device=('cuda' if HAS_CUDA else 'cpu'),
                  random_state=SEED, n_jobs=-1, max_bin=256,
                  n_estimators=600 if MODE=='full' else 350,
                  max_depth=6, learning_rate=0.05,
                  subsample=0.9, colsample_bytree=0.8, reg_lambda=1.0)
    if base_params: params.update(base_params)
    return xgb.XGBRegressor(**params)

def objective(trial, Xtr, ytr, Xva, yva):
    params = {'n_estimators': trial.suggest_int('n_estimators', 250, 900),
              'max_depth': trial.suggest_int('max_depth', 4, 8),
              'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.15, log=True),
              'subsample': trial.suggest_float('subsample', 0.7, 1.0),
              'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
              'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 3.0, log=True),
              'tweedie_variance_power': trial.suggest_float('tweedie_variance_power', 1.1, 1.9),
              'max_bin': trial.suggest_int('max_bin', 128, 512),
              'tree_method': 'hist', 'device': ('cuda' if HAS_CUDA else 'cpu'),
              'objective': 'reg:tweedie', 'random_state': SEED, 'n_jobs': -1}
    model = xgb.XGBRegressor(**params)
    model.fit(Xtr, ytr, eval_set=[(Xva, yva)], verbose=False)
    pred = model.predict(Xva)
    return float(mean_absolute_error(yva, pred))

def run_optuna(name, Xtr, ytr, Xva, yva, n_trials):
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=SEED))
    study.optimize(lambda t: objective(t, Xtr, ytr, Xva, yva), n_trials=n_trials, show_progress_bar=False)
    save_json(study.best_params, OUTDIR/f'optuna_tweedie_{name.lower()}.json')
    return study.best_params

with timer('Precompute preprocess for Tweedie'):
    Xt_train = preprocess.fit_transform(X_train, y_train_log).astype('float32')
    Xt_valid = preprocess.transform(X_valid).astype('float32')

y_prop_tr = sanitize_targets_nat(y_train[:,0]); y_prop_va = sanitize_targets_nat(y_valid[:,0])
y_crop_tr = sanitize_targets_nat(y_train[:,1]); y_crop_va = sanitize_targets_nat(y_valid[:,1])

TRIALS = 12 if MODE in ('2h','3h') else 30
best_prop = run_optuna('Property', Xt_train, y_prop_tr, Xt_valid, y_prop_va, n_trials=TRIALS)
best_crop = run_optuna('Crop', Xt_train, y_crop_tr, Xt_valid, y_crop_va, n_trials=TRIALS)

xgb_prop = make_xgb_tweedie(best_prop); xgb_prop.fit(Xt_train, y_prop_tr, eval_set=[(Xt_valid, y_prop_va)], verbose=False)
xgb_crop = make_xgb_tweedie(best_crop); xgb_crop.fit(Xt_train, y_crop_tr, eval_set=[(Xt_valid, y_crop_va)], verbose=False)

def eval_metrics(y_true, y_pred):
    return {'MAE': float(mean_absolute_error(y_true, y_pred)),
            'RMSE': float(np.sqrt(mean_squared_error(y_true, y_pred))),
            'R2': float(r2_score(y_true, y_pred))}

metrics = {'xgb_prop': eval_metrics(y_prop_va, xgb_prop.predict(Xt_valid)),
           'xgb_crop': eval_metrics(y_crop_va, xgb_crop.predict(Xt_valid))}
save_json(metrics, OUTDIR/'metrics_xgb_tweedie.json')
save_joblib(xgb_prop, OUTDIR/'xgb_tweedie_property.joblib')
save_joblib(xgb_crop, OUTDIR/'xgb_tweedie_crop.joblib')

print('✅ Saved tuned XGB Tweedie models and metrics to results/')


In [ ]:

# === 6) Baseline Ridge & XGB (context, fast) ===
from sklearn.linear_model import Ridge
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def metrics_frame(y_true, y_pred, labels):
    out = {}
    for i, name in enumerate(labels):
        out[name] = dict(
            MAE=float(mean_absolute_error(y_true[:, i], y_pred[:, i])),
            RMSE=float(np.sqrt(mean_squared_error(y_true[:, i], y_pred[:, i]))),
            R2=float(r2_score(y_true[:, i], y_pred[:, i])),
        )
    return pd.DataFrame(out).T

# Precompute features for multi-target models
with timer("Precompute preprocess on train/valid for baselines"):
    Xt_train_mt = Xt_train
    Xt_valid_mt = Xt_valid

def make_ridge(alpha=3.0): 
    return Ridge(alpha=alpha, fit_intercept=True, random_state=SEED)

def make_xgb_fast():
    return xgb.XGBRegressor(
        n_estimators=350 if MODE!='full' else 700,
        max_depth=6, learning_rate=0.07 if MODE!='full' else 0.05,
        subsample=0.9, colsample_bytree=0.9, random_state=SEED,
        tree_method='hist', device=('cuda' if HAS_CUDA else 'cpu'), n_jobs=-1
    )

# Fit 2 independent targets for each baseline (simple approach)
def fit_two_targets(builder):
    m_prop = builder(); m_crop = builder()
    m_prop.fit(Xt_train_mt, y_train[:,0]); m_crop.fit(Xt_train_mt, y_train[:,1])
    p_prop = m_prop.predict(Xt_valid_mt); p_crop = m_crop.predict(Xt_valid_mt)
    Yp = np.column_stack([p_prop, p_crop])
    return Yp

ridge_pred = fit_two_targets(make_ridge)
xgb_pred = fit_two_targets(make_xgb_fast)

mf_ridge = metrics_frame(y_valid, ridge_pred, ['damage_property','damage_crops'])
mf_xgb = metrics_frame(y_valid, xgb_pred, ['damage_property','damage_crops'])

save_json({'ridge': mf_ridge.to_dict(orient='index'),
           'xgb'  : mf_xgb.to_dict(orient='index')},
          OUTDIR/'validation_metrics_first_pass.json')

print("\n📊 Baseline (validation) results:")
print("Ridge:\n", mf_ridge)
print("XGB  :\n", mf_xgb)
print("✅ Saved baseline metrics to results/validation_metrics_first_pass.json")


In [ ]:

# === 7) Quantile XGB (Intervals) ===
import xgboost as xgb

QUANTILES = [0.1, 0.5, 0.9]

def make_quantile_xgb(q):
    def quantile_obj(preds, dtrain):
        y = dtrain.get_label()
        e = y - preds
        grad = np.where(e>0, -q, 1-q)
        hess = np.ones_like(grad) * 1e-6
        return grad, hess
    def quantile_eval(preds, dtrain):
        y = dtrain.get_label()
        e = y - preds
        loss = np.where(e>=0, q*e, (q-1)*e)
        return 'q_pinball', float(np.mean(loss))
    class XGBQuantile(xgb.XGBRegressor):
        def fit(self, X, y):
            dtrain = xgb.DMatrix(X, label=y)
            self._Booster = xgb.train(
                params={'max_depth': 6, 'eta': 0.05, 'subsample': 0.9, 'colsample_bytree': 0.9,
                        'lambda': 1.0, 'tree_method': 'hist', 'verbosity': 0},
                dtrain=dtrain, num_boost_round=(600 if MODE=='full' else 400),
                obj=quantile_obj, feval=quantile_eval, verbose_eval=False
            ); return self
        def predict(self, X): return self._Booster.predict(xgb.DMatrix(X))
    return XGBQuantile()

if 'Xt_test' not in globals():
    Xt_test = preprocess.transform(X_test).astype(np.float32, copy=False)

# Use log targets for quantile on log-scale, then expm1
ytr_p = y_train_log[:,0]; yva_p = y_valid_log[:,0]
ytr_c = y_train_log[:,1]; yva_c = y_valid_log[:,1]

quantile_models_xgb = {'prop':{}, 'crop':{}}
preds_valid_q = {'prop':{}, 'crop':{}}
for q in QUANTILES:
    for target, (ytr, yva) in {'prop': (ytr_p, yva_p),
                               'crop': (ytr_c, yva_c)}.items():
        model = make_quantile_xgb(q); model.fit(Xt_train, ytr)
        preds_valid_q[target][q] = model.predict(Xt_valid)
        quantile_models_xgb[target][q] = model

for q in QUANTILES:
    for target, yv_log in {'prop': yva_p, 'crop': yva_c}.items():
        pv = preds_valid_q[target][q]
        diff = np.expm1(yv_log) - np.expm1(pv)
        pin = float(np.mean(np.where(diff>=0, q*diff, (q-1)*diff)))
        print(f"q={q:.1f} {target}: pinball={pin:,.2f}")

preds_test = {}
for target in ['prop','crop']:
    preds_test[target] = {q: m.predict(Xt_test) for q,m in quantile_models_xgb[target].items()}

interval_df = pd.DataFrame({
    'prop_p10': np.expm1(preds_test['prop'][0.1]), 'prop_p50': np.expm1(preds_test['prop'][0.5]), 'prop_p90': np.expm1(preds_test['prop'][0.9]), 'prop_true': np.expm1(y_test_log[:,0]),
    'crop_p10': np.expm1(preds_test['crop'][0.1]), 'crop_p50': np.expm1(preds_test['crop'][0.5]), 'crop_p90': np.expm1(preds_test['crop'][0.9]), 'crop_true': np.expm1(y_test_log[:,1]),
})
interval_df.to_csv(OUTDIR/'test_quantile_predictions.csv', index=False); print("✅ Saved:", OUTDIR/'test_quantile_predictions.csv')


In [ ]:

# === 8) SHAP Analysis (decoded features) ===
import shap, matplotlib.pyplot as plt

# Manually construct feature names (numeric + cat OHE + text embeddings)
feature_names = []

for c in num_cols:
    feature_names.append(f"num__{c}")
for c in cat_cols:
    feature_names.append(f"cat__{c}_OHE")  # TargetEncoder alt would be _te

embed_dim = 384  # all-MiniLM-L6-v2
try:
    embed_dim = embed_model.get_sentence_embedding_dimension()
except Exception:
    pass

for c in text_cols:
    for i in range(embed_dim):
        feature_names.append(f"text__{c}_emb_{i}")

np.save(OUTDIR/'feature_names.npy', np.array(feature_names, dtype=object))

# Use tuned property model (as example) with a small sample for speed
sample_n = min(2000, Xt_valid.shape[0])
Xs = Xt_valid[:sample_n]
explainer = shap.Explainer(xgb_prop)  # tree explainer
sv = explainer(Xs, check_additivity=False)

np.save(OUTDIR/'shap_values.npy', sv.values)
np.save(OUTDIR/'shap_sample.npy', Xs)

# Global bars
plt.figure(); shap.plots.bar(sv, max_display=20, show=False); plt.tight_layout()
plt.savefig(OUTDIR/'shap_bar.png', dpi=160, bbox_inches='tight'); plt.close()

# Beeswarm
plt.figure(); shap.plots.beeswarm(sv, max_display=20, show=False); plt.tight_layout()
plt.savefig(OUTDIR/'shap_beeswarm.png', dpi=160, bbox_inches='tight'); plt.close()

print("✅ Saved SHAP artifacts to results/")


In [ ]:

# === 9) Interval Diagnostics (coverage & reliability) ===
import matplotlib.pyplot as plt

df_int = pd.read_csv(OUTDIR/'test_quantile_predictions.csv')
def cov_rate(y, lo, hi): 
    return float(np.mean((y >= lo) & (y <= hi)))

prop_cov = cov_rate(df_int['prop_true'], df_int['prop_p10'], df_int['prop_p90'])
crop_cov = cov_rate(df_int['crop_true'], df_int['crop_p10'], df_int['crop_p90'])

# Bar plot
fig = plt.figure()
plt.bar(['Property','Crops'], [prop_cov, crop_cov])
plt.axhline(0.8, linestyle='--')
plt.title('80% Interval Coverage (p10–p90)')
plt.ylim(0,1); plt.tight_layout()
plt.savefig(OUTDIR/'quantile_coverage_bar.png', dpi=160, bbox_inches='tight'); plt.close(fig)

# Reliability (quantile calibration diagnostic)
qs = [0.1, 0.5, 0.9]
def quantile_reliability(y_true, preds_dict):
    pts = []
    for q in qs:
        yhat = preds_dict[q]
        # fraction below predicted quantile ~ q if well calibrated
        frac = float(np.mean(y_true <= yhat))
        pts.append((q, frac))
    return pts

prop_pts = quantile_reliability(df_int['prop_true'], {q: df_int[f'prop_p{int(q*100)}'] for q in qs})
crop_pts = quantile_reliability(df_int['crop_true'], {q: df_int[f'crop_p{int(q*100)}'] for q in qs})

fig = plt.figure()
plt.plot([0,1],[0,1],'--')
plt.scatter([p[0] for p in prop_pts], [p[1] for p in prop_pts], label='Property')
plt.scatter([p[0] for p in crop_pts], [p[1] for p in crop_pts], label='Crops')
plt.legend(); plt.xlabel('Nominal quantile'); plt.ylabel('Observed fraction'); plt.title('Quantile Reliability')
plt.tight_layout(); plt.savefig(OUTDIR/'quantile_intervals_reliability.png', dpi=160, bbox_inches='tight'); plt.close(fig)

print(f"Coverage — property: {prop_cov:.3f}, crops: {crop_cov:.3f}")
print('✅ Saved interval diagnostics images to results/')


In [ ]:

# === 10) Export predict.py helper (batch predictions) ===
script = '''#!/usr/bin/env python3
import sys, pandas as pd, numpy as np, joblib
def main(inp,out):
    bprop=joblib.load("results/xgb_tweedie_property.joblib")
    bcrop=joblib.load("results/xgb_tweedie_crop.joblib")
    pre=joblib.load("results/preprocess.joblib")
    df=pd.read_csv(inp)
    X=pre.transform(df)
    p_prop=bprop.predict(X); p_crop=bcrop.predict(X)
    pd.DataFrame({"pred_damage_property":p_prop,"pred_damage_crops":p_crop}).to_csv(out,index=False)
if __name__=="__main__": main(sys.argv[1], sys.argv[2])
'''
p = OUTDIR/'predict.py'
with open(p,'w', encoding='utf-8') as f: f.write(script)
print("✅ Wrote", p)


In [ ]:

# === 11) Save preprocess artifact ===
import joblib
joblib.dump(preprocess, OUTDIR/'preprocess.joblib')
print("✅ Saved:", OUTDIR/'preprocess.joblib')
